In [1]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)
import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

In [3]:
texts_with_elias = [
    "selected-data/elias/message_1.json",
]
texts_with_petyo = [
    "selected-data/petyo/message_1.json",
    "selected-data/petyo/message_2.json",
    "selected-data/petyo/message_3.json",
    "selected-data/petyo/message_4.json",
    "selected-data/petyo/message_5.json",
]
texts_with_others_dict = {
    "anna": ["selected-data/others/anna.json"],
    "patryk": ["selected-data/others/patryk.json"],
    "andreas": ["selected-data/others/andreas.json"],
    "victoria": ["selected-data/others/victoria.json", "selected-data/others/victoria2.json"],
    "joanna": ["selected-data/others/joanna.json"],
    "antoni": ["selected-data/others/antoni.json"],
    "arijan": ["selected-data/others/arijan.json"],
    "denis": ["selected-data/others/denis.json"],
    "alexandra": ["selected-data/others/alexandra.json"],
    "FED": ["selected-data/others/FED.json"],
    "filip": ["selected-data/others/filip.json"],
    "kuba": ["selected-data/others/kuba.json"],
    "laura": ["selected-data/others/laura.json"],
    "liisa": ["selected-data/others/liisa.json"],
    "luiza": ["selected-data/others/luiza.json"],
    "marcus": ["selected-data/others/marcus.json"],
}


et = persona.PersonaEncoder()
et.parse_fb_messages(texts_with_elias, "elias")
et.parse_fb_messages(texts_with_petyo, "petyo")
for name, texts in texts_with_others_dict.items():
    et.parse_fb_messages(texts, name)

# et.parse_rosebud_entries("selected-data/rosebud.md", "rosebud")
et.filter_chats_empty()
et.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)


for nameid, chat in et.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Airidas Brikas" else "Friend"

# Start all chats from 2/3rds
# for nameid, chat in et.chats.items():
#     et.chats[nameid] = chat[int(len(chat)/3 * 2):]

# et.select_chat_limited_by_tokens("elias", 6000)
# et.select_chat_limited_by_tokens("petyo", 6000)
et.select_chat_full("elias")
et.select_chat_full("petyo")

for name in texts_with_others_dict.keys():
    et.select_chat_full(name)

# et.select_nonChat_module_full("rosebud")

token_counts = et.count_all_selected_chat_tokens() # token_counts used later for statistics
print(f"Combined tokens: {sum(token_counts.values())}")

Read 1946 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-13 to 2024-03-06
Messages saved to self.chats['elias']
Read 40036 messages from 5 files. Failed to read 0 messages.
Messages ranged from 2020-08-17 to 2024-03-04
Messages saved to self.chats['petyo']
Read 7953 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2023-05-12 to 2024-03-04
Messages saved to self.chats['anna']
Read 5734 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2020-09-18 to 2024-03-02
Messages saved to self.chats['patryk']
Read 372 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-03-19 to 2024-02-24
Messages saved to self.chats['andreas']
Read 3399 messages from 2 files. Failed to read 0 messages.
Messages ranged from 2021-08-23 to 2024-03-02
Messages saved to self.chats['victoria']
Read 2951 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-11-13 to 2024-02-19
Messages saved to self.ch

# Final Boss

### Gen embeddings

In [20]:
CHUNK_SIZES = [75]
chunk_size = CHUNK_SIZES[0]
EMBED_MODEL = "nomic-embed-text"
OVERLAP_SIZES = [3]
overlap_size = OVERLAP_SIZES[0]

chunks = []
chunk_token_counts = []
for chat in et.selectedChats.values():
    messages = list(chat)  # Convert chat iterator to list for easier slicing
    num_messages = len(messages)
    for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
        chunk = messages[i:i + chunk_size]  # Extract chunk of messages
        chunk_text = "\\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
        chunks.append(chunk_text)  # Append chunk to list of chunks
        chunk_token_counts.append(utils.count_tokens(chunk_text))  # Append token count of the chunk
avg_chunk_token_count = sum(chunk_token_counts) / len(chunk_token_counts)
embeddings = []
progress, chunks_len = 0, len(chunks)
for chunk_text in chunks:
    progress += 1
    print(f"\rChunk {progress}/{chunks_len}", end="")
    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)

Chunk 929/929

In [31]:
EMBEDDING_NAMEID = "airidas_finalboss_1"
AUTO_INFO = {
    "model": EMBED_MODEL,
    "CHUNK_SIZE": chunk_size,
    "OVERLAP_SIZE": overlap_size,
    "chunks_count": len(chunks),
    "modules_chat": token_counts,
    "overlap_size": overlap_size,
}

bu.quickJSON(AUTO_INFO, f"embeddings/{EMBEDDING_NAMEID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"embeddings/{EMBEDDING_NAMEID}_embeddings.json")

### Load embeddings from file

In [6]:
EMBEDDING_NAMEID = "airidas_finalboss_1"

import json
with open(f"embeddings/{EMBEDDING_NAMEID}_embeddings.json", "r") as f:
    data = json.load(f)
    chunks = data["chunks"]
    embeddings = data["embeddings"]

try:
    with open(f"embeddings/{EMBEDDING_NAMEID}_info.json", "r") as f:
        AUTO_INFO = json.load(f)
        try:
            EMBED_MODEL = AUTO_INFO["model"]
            chunk_size = AUTO_INFO["CHUNK_SIZE"]
        except KeyError:
            print("WARNING: Info text does not contain model information")
except:
    print("WARNING: No Info file found. Make sure embedding model is matching.")

print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

Chunks:929, embeds:929


### The actual boss

In [10]:
PROMPT_METHOD ="IMPERSONATE"
SUBJECT = "airidas"
RETRIAVAL_METHODS = ["static"]#, "hybrid", "dynamic", 
NUM_RUNS = 3
# MODEL = "llama3-70b"
# MODEL = "llama3-8b"
MODEL = "mixtral-8x22b"
# max_tokens = [7500]
max_tokens = [0, 4000, 7500]

survs = [survey.KanoSurvey(), survey.PersonalitySurvey()]
for surv in survs:
    if isinstance(surv, survey.KanoSurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 40
        SURVEY_TYPE = "KanoSurvey",
        WHICH_SURVEY = "kano"
        RETRIEVAL_PROMPT = "video game features"
        SURVEY = "video game preferences"
        METHOD = "a Kano survey"
    elif isinstance(surv, survey.PersonalitySurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 50
        SURVEY_TYPE = "PersonalitySurvey",
        WHICH_SURVEY = "pers"
        RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism"
        SURVEY = "personality traits"
        METHOD = "an OCEAN test"

    for max_token in max_tokens:
        for RETRIAVAL_METHOD in RETRIAVAL_METHODS:
            if RETRIAVAL_METHOD == "dynamic":
                dynamic_retrieval_prompts = list(surv.questions)
                dynamic_chunks_most_similar: List[List[str]] = [] 
                progress = 0
                lenn = len(dynamic_retrieval_prompts)

                # Dynamic Retrieval
                for prompt in dynamic_retrieval_prompts:
                    progress += 1
                    print(f"\rPrompt {progress}/{lenn}", end="")

                    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]

                    ## Chunking and Cosine Similarity
                    max_chunks_count = int((max_token / avg_chunk_token_count))
                    chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)
                    chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]

                    # Limit chunks by tokens
                    if max_token == 0:
                        chunks_most_similar = chunks_most_similar[:1]
                    else:
                        # Limit chunks by man token count
                        cur_tc = 0 # current token count
                        selected_chunks = []
                        for chunk in chunks_most_similar:
                            tk_in_chunk = utils.count_tokens(chunk)
                            if cur_tc + tk_in_chunk >= max_token:
                                break
                            cur_tc += tk_in_chunk
                            selected_chunks.append(chunk)
                        chunks_most_similar = selected_chunks
                    
                    # Finalize chunks
                    tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                    dynamic_chunks_most_similar.append(chunks_most_similar)
                print(end="\n")

                # Count total tokens in all chunks
                tokens_in_chunks = 0
                for chunks_most_similar in dynamic_chunks_most_similar:
                    for chunk in chunks_most_similar:
                        tokens_in_chunks += utils.count_tokens(chunk)
                # del chunks_most_similar_embeddings  # free memory
                print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
                final_prompts = []
                prompt_template = """
for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famous people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n",
            "\\n\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),     
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """

            elif RETRIAVAL_METHOD == "static":
                prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
                chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)
                chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]
                ## Chunking
                if max_token == 0:
                    chunks_most_similar = chunks_most_similar[:1]
                else:
                    # Limit chunks by man token count
                    cur_tc = 0 # current token count
                    selected_chunks = []
                    for chunk in chunks_most_similar:
                        tk_in_chunk = utils.count_tokens(chunk)
                        if cur_tc + tk_in_chunk >= max_token:
                            break
                        cur_tc += tk_in_chunk
                        selected_chunks.append(chunk)
                    chunks_most_similar = selected_chunks
                
                # Count total tokens in all chunks
                tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                print(f"Tokens in average chunk group: {tokens_in_chunks/len(chunks_most_similar)}")

                del chunks_most_similar_embeddings # free memory
                #Retrieve^
                final_prompts = []
                prompt_template = """
for question in surv.questions:
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famous people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n",
            "\\n\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),     
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """
            else: raise ValueError(f"Unknown retrieval method: {RETRIAVAL_METHOD}")
            exec(prompt_template)
            SIM_ID = f"{SUBJECT}-{WHICH_SURVEY}-{RETRIAVAL_METHOD}-{max_token}_{MODEL}_V8"
            bu.quickJSON(final_prompts, f"temp/{SIM_ID}_prompts.json")
            for num_run in range(NUM_RUNS):
                instructions = {
                    "prompt_file": f"batch/prompts/{SIM_ID}_prompts.json",
                    "survey_type": f"{SURVEY_TYPE[0]}",
                    "isLocal": True,
                    "LIMIT": None
                }
                settings = {
                    "model": MODEL,
                    "timeout": 300}
                AUTO_INFO = {
                    "CHUNK_SIZE": chunk_size,
                    "OVERLAP_SIZE": overlap_size,
                    "CTX_limit": max_token,
                    "chunk_count": len(chunks_most_similar),
                    "EMBED_MODEL": EMBED_MODEL,
                    "prompt method": PROMPT_METHOD,
                    "retrieval method": RETRIAVAL_METHOD,
                    "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
                    "prompt_count": PROMPT_COUNT,
                    "survey": WHICH_SURVEY,
                    "SUBJECT": SUBJECT,
                    "prompt_template": prompt_template,
                    **utils.describe_prompts(final_prompts)
                }
                bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"temp/runs/{SIM_ID}_{num_run}.json")

Using default Kano Survey CSV file: surveys/survey_kano-model.csv
Using default Personality Survey CSV file: surveys/survey_personality-test.csv
Tokens in average chunk group: 834.0
Tokens in average chunk group: 872.0
Tokens in average chunk group: 988.2857142857143
Tokens in average chunk group: 950.0
Tokens in average chunk group: 1332.6666666666667
Tokens in average chunk group: 1195.5


In [ ]:
PROMPT_METHOD ="IMPERSONATE"
SUBJECT = "airidas"
RETRIAVAL_METHODS = ["dynamic"]#, "hybrid"]
NUM_RUNS = 3
# MODEL = "llama3-70b"
# MODEL = "llama3-8b"
MODEL = "mixtral-8x22b"
max_tokens = [0, 4000, 7500]
# max_tokens = [0, 4000, 15500]

survs = [survey.KanoSurvey(), survey.PersonalitySurvey()]
for surv in survs:
    if isinstance(surv, survey.KanoSurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 40
        SURVEY_TYPE = "KanoSurvey",
        WHICH_SURVEY = "kano"
        RETRIEVAL_PROMPT = "video game features"
        SURVEY = "video game preferences"
        METHOD = "a Kano survey"
    elif isinstance(surv, survey.PersonalitySurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 50
        SURVEY_TYPE = "PersonalitySurvey",
        WHICH_SURVEY = "pers"
        RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism"
        SURVEY = "personality traits"
        METHOD = "an OCEAN test"

    for max_token in max_tokens:

        for RETRIAVAL_METHOD in RETRIAVAL_METHODS:
            if RETRIAVAL_METHOD == "dynamic":
                dynamic_retrieval_prompts = list(surv.questions)
                dynamic_chunks_most_similar: List[List[str]] = [] 
                progress = 0
                lenn = len(dynamic_retrieval_prompts)
                for prompt in dynamic_retrieval_prompts:
                    progress += 1
                    print(f"\rPrompt {progress}/{lenn}", end="")
                    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]

                    ## Chunking
                    chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)
                    chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]

                    if max_token == 0:
                        chunks_most_similar = chunks_most_similar[:1]
                    else:
                        # Limit chunks by man token count
                        cur_tc = 0 # current token count
                        selected_chunks = []
                        for chunk in chunks_most_similar:
                            tk_in_chunk = utils.count_tokens(chunk)
                            if cur_tc + tk_in_chunk >= max_token:
                                break
                            cur_tc += tk_in_chunk
                            selected_chunks.append(chunk)
                        chunks_most_similar = selected_chunks
                    ##

                    tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                    dynamic_chunks_most_similar.append(chunks_most_similar)
                print(end="\n")
                tokens_in_chunks = 0
                for chunks_most_similar in dynamic_chunks_most_similar:
                    for chunk in chunks_most_similar:
                        tokens_in_chunks += utils.count_tokens(chunk)
                del chunks_most_similar_embeddings  # free memory
                print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
                final_prompts = []
                prompt_template = """
for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famous people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n",
            "\\n\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),     
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """
            else: print("not dynamic")
            exec(prompt_template)

            # Save prompts
            SIM_ID = f"{SUBJECT}-{WHICH_SURVEY}-{str(max_token)}_{MODEL}_V7"
            bu.quickJSON(final_prompts, f"temp/{SIM_ID}_prompts.json")

            
            for num_run in range(NUM_RUNS):
                instructions = {
                    "prompt_file": f"batch/prompts/{SIM_ID}_prompts.json",
                    "survey_type": f"{SURVEY_TYPE[0]}",
                    "isLocal": True,
                    "LIMIT": None
                }
                settings = {
                    "model": MODEL,
                    "timeout": 300}
                AUTO_INFO = {
                    "CHUNK_SIZE": chunk_size,
                    "OVERLAP_SIZE": overlap_size,
                    "CTX_limit": max_token,
                    "chunk_count": len(chunks_most_similar),
                    "EMBED_MODEL": EMBED_MODEL,
                    "prompt method": PROMPT_METHOD,
                    "retrieval method": RETRIAVAL_METHOD,
                    "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
                    "prompt_count": PROMPT_COUNT,
                    "survey": WHICH_SURVEY,
                    "SUBJECT": SUBJECT,
                    "prompt_template": prompt_template,
                    **utils.describe_prompts(final_prompts)
                }
                bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"temp/runs/{SIM_ID}_{num_run}.json")

In [6]:
chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)

In [18]:
max_tokens = 8000
cur_tc = 0 # current token count
selected_chunks = []
for chunk in chunks_most_similar:
    tk_in_chunk = utils.count_tokens(chunk)
    if cur_tc + tk_in_chunk >= max_tokens:
        break
    cur_tc += tk_in_chunk
    selected_chunks.append(chunk)
print(f"Tokens in chunks: {cur_tc}")
len(selected_chunks)

Tokens in chunks: 6947


7